In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1748092706331_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

## Importar librerías

In [55]:
#import pandas as pd
#import numpy as np
#from sklearn.preprocessing import StandardScaler
from pyspark.sql.functions import when, col, isnan, count, avg, broadcast
from pyspark.sql.window import Window
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import expr

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Cargar dataset desde S3

In [56]:
# Cargar dataset
# desde S3
data_raw=spark.read.csv('s3://aarangog2proyectointegrador/zona raw/who_life_exp.csv',inferSchema=True,header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
data_raw.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['country', 'country_code', 'region', 'year', 'life_expect', 'life_exp60', 'adult_mortality', 'infant_mort', 'age1-4mort', 'alcohol', 'bmi', 'age5-19thinness', 'age5-19obesity', 'hepatitis', 'measles', 'polio', 'diphtheria', 'basic_water', 'doctors', 'hospitals', 'gni_capita', 'gghe-d', 'che_gdp', 'une_pop', 'une_infant', 'une_life', 'une_hiv', 'une_gni', 'une_poverty', 'une_edu_spend', 'une_literacy', 'une_school']

In [58]:
print((data_raw.count(),len(data_raw.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(3111, 32)

## Generación de data_filtered

In [59]:
# Reorganizar las columnas
columnas = ["region", "country", "country_code", "year", "une_pop", "gni_capita", "une_gni", "gghe-d",
            "che_gdp", "une_edu_spend", "une_literacy", "une_school", "une_poverty", "basic_water", "doctors",
            "hospitals", "bmi", "age5-19thinness", "age5-19obesity", "alcohol", "hepatitis", "measles", "polio",
            "diphtheria", "une_hiv", "adult_mortality", "infant_mort", "une_infant", "age1-4mort", "life_expect",
            "une_life", "life_exp60"]

data_filtered = data_raw.select(*columnas)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
data_filtered.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['region', 'country', 'country_code', 'year', 'une_pop', 'gni_capita', 'une_gni', 'gghe-d', 'che_gdp', 'une_edu_spend', 'une_literacy', 'une_school', 'une_poverty', 'basic_water', 'doctors', 'hospitals', 'bmi', 'age5-19thinness', 'age5-19obesity', 'alcohol', 'hepatitis', 'measles', 'polio', 'diphtheria', 'une_hiv', 'adult_mortality', 'infant_mort', 'une_infant', 'age1-4mort', 'life_expect', 'une_life', 'life_exp60']

In [61]:
# Limitar valores de "basic_water" a 100
data_filtered.filter(col("basic_water") > 100).show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------------+----+------------------+----------+-------+-----------------+-----------------+-----------------+------------+----------+-----------+-----------+-------+---------+----+---------------+--------------+-------+---------+-------+-----+----------+-------+---------------+---------------------+----------+----------+-----------------+--------+------------------+
|region|country|country_code|year|une_pop           |gni_capita|une_gni|gghe-d           |che_gdp          |une_edu_spend    |une_literacy|une_school|une_poverty|basic_water|doctors|hospitals|bmi |age5-19thinness|age5-19obesity|alcohol|hepatitis|measles|polio|diphtheria|une_hiv|adult_mortality|infant_mort          |une_infant|age1-4mort|life_expect      |une_life|life_exp60        |
+------+-------+------------+----+------------------+----------+-------+-----------------+-----------------+-----------------+------------+----------+-----------+-----------+-------+---------+----+---------------+--------------+

In [62]:
data_filtered = data_filtered.withColumn("basic_water", when(col("basic_water") > 100, 100).otherwise(col("basic_water")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
data_filtered.filter(col("basic_water") > 100).show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------------+----+-------+----------+-------+------+-------+-------------+------------+----------+-----------+-----------+-------+---------+---+---------------+--------------+-------+---------+-------+-----+----------+-------+---------------+-----------+----------+----------+-----------+--------+----------+
|region|country|country_code|year|une_pop|gni_capita|une_gni|gghe-d|che_gdp|une_edu_spend|une_literacy|une_school|une_poverty|basic_water|doctors|hospitals|bmi|age5-19thinness|age5-19obesity|alcohol|hepatitis|measles|polio|diphtheria|une_hiv|adult_mortality|infant_mort|une_infant|age1-4mort|life_expect|une_life|life_exp60|
+------+-------+------------+----+-------+----------+-------+------+-------+-------------+------------+----------+-----------+-----------+-------+---------+---+---------------+--------------+-------+---------+-------+-----+----------+-------+---------------+-----------+----------+----------+-----------+--------+----------+
+------+-------+---------

In [64]:
# Guardar en S3
write_uri='s3://aarangog2proyectointegrador/zona trusted/data_filtered'

data_filtered.write.format('parquet').save(write_uri)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Generación de data_selected

In [65]:
# Eliminar columnas
columnas_a_eliminar = ["une_life","une_infant","gni_capita","hospitals","une_literacy",
                       "une_school","une_poverty", "life_exp60","infant_mort", "adult_mortality", "age1-4mort"]
data_selected = data_filtered.drop(*columnas_a_eliminar)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
data_selected.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['region', 'country', 'country_code', 'year', 'une_pop', 'une_gni', 'gghe-d', 'che_gdp', 'une_edu_spend', 'basic_water', 'doctors', 'bmi', 'age5-19thinness', 'age5-19obesity', 'alcohol', 'hepatitis', 'measles', 'polio', 'diphtheria', 'une_hiv', 'life_expect']

In [ ]:
# Filtrar países con exceso de datos faltantes
countries_to_remove = ["South Sudan", "Somalia", "Democratic People's Republic of Korea", "Sudan"]
data_selected = data_selected.filter(~col("country").isin(countries_to_remove))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
print((data_selected.count(),len(data_selected.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(3043, 21)

In [69]:
# Guardar en S3
write_uri='s3://aarangog2proyectointegrador/zona trusted/data_selected'

data_selected.write.format('parquet').save(write_uri)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Generación de data_imputed

In [70]:
data_selected.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data_selected.columns]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------------+----+-------+-------+------+-------+-------------+-----------+-------+---+---------------+--------------+-------+---------+-------+-----+----------+-------+-----------+
|region|country|country_code|year|une_pop|une_gni|gghe-d|che_gdp|une_edu_spend|basic_water|doctors|bmi|age5-19thinness|age5-19obesity|alcohol|hepatitis|measles|polio|diphtheria|une_hiv|life_expect|
+------+-------+------------+----+-------+-------+------+-------+-------------+-----------+-------+---+---------------+--------------+-------+---------+-------+-----+----------+-------+-----------+
|     0|      0|           0|   0|     37|     72|    49|     66|         1230|         21|   1276|  0|              0|             0|     22|      533|      8|    8|         8|    724|          0|
+------+-------+------------+----+-------+-------+------+-------+-------------+-----------+-------+---+---------------+--------------+-------+---------+-------+-----+----------+-------+-----------+

In [71]:
# Interpolación aproximada por país para columnas con nulos (simulado por promedio por país)

# Detectar columnas con nulos
columns_with_nan = [c for c in data_selected.columns if data_selected.filter(col(c).isNull()).count() > 0]

# Inicializar data_imputed con data_selected
data_imputed_1 = data_selected

# Imputar promedio por país
for c in columns_with_nan:
    window_spec = Window.partitionBy("country")
    data_imputed_1 = data_imputed_1.withColumn(
        c,
        when(col(c).isNull(), avg(c).over(window_spec)).otherwise(col(c))
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
data_imputed_1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data_imputed_1.columns]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------------+----+-------+-------+------+-------+-------------+-----------+-------+---+---------------+--------------+-------+---------+-------+-----+----------+-------+-----------+
|region|country|country_code|year|une_pop|une_gni|gghe-d|che_gdp|une_edu_spend|basic_water|doctors|bmi|age5-19thinness|age5-19obesity|alcohol|hepatitis|measles|polio|diphtheria|une_hiv|life_expect|
+------+-------+------------+----+-------+-------+------+-------+-------------+-----------+-------+---+---------------+--------------+-------+---------+-------+-----+----------+-------+-----------+
|     0|      0|           0|   0|      0|     51|    17|     34|          204|          0|      0|  0|              0|             0|      0|      153|      0|    0|         0|    714|          0|
+------+-------+------------+----+-------+-------+------+-------+-------------+-----------+-------+---+---------------+--------------+-------+---------+-------+-----+----------+-------+-----------+

In [73]:
# Lista de columnas que tienen valores nulos y queremos imputar con promedio por región
cols_a_imputar = ['une_gni', 'gghe-d', 'che_gdp', 'une_edu_spend', 'hepatitis', 'une_hiv']

# Renombramos columnas antes del join para evitar conflicto de nombres
promedios_region = (
    data_imputed_1
    .groupBy("region")
    .agg(*[avg(c).alias(f"{c}_avg") for c in cols_a_imputar])
)

# Realiza el join de manera segura (usamos alias para evitar conflicto directo con columnas originales)
data_imputed = (
    data_imputed_1.alias("data")
    .join(broadcast(promedios_region).alias("prom"), on="region", how="left")
)

# Imputamos valores nulos con promedio por región
for c in cols_a_imputar:
    data_imputed = data_imputed.withColumn(
        c,
        when(col(f"data.{c}").isNull(), col(f"prom.{c}_avg")).otherwise(col(f"data.{c}"))
    )

# Si ya no necesitas las columnas auxiliares, puedes seleccionar solo las columnas originales
# (ya imputadas)
columnas_finales = data_selected.columns
data_imputed = data_imputed.select(*[col(c) for c in columnas_finales])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
data_imputed.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data_imputed.columns]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------------+----+-------+-------+------+-------+-------------+-----------+-------+---+---------------+--------------+-------+---------+-------+-----+----------+-------+-----------+
|region|country|country_code|year|une_pop|une_gni|gghe-d|che_gdp|une_edu_spend|basic_water|doctors|bmi|age5-19thinness|age5-19obesity|alcohol|hepatitis|measles|polio|diphtheria|une_hiv|life_expect|
+------+-------+------------+----+-------+-------+------+-------+-------------+-----------+-------+---+---------------+--------------+-------+---------+-------+-----+----------+-------+-----------+
|     0|      0|           0|   0|      0|      0|     0|      0|            0|          0|      0|  0|              0|             0|      0|        0|      0|    0|         0|      0|          0|
+------+-------+------------+----+-------+-------+------+-------+-------------+-----------+-------+---+---------------+--------------+-------+---------+-------+-----+----------+-------+-----------+

In [75]:
# Guardar en S3
write_uri='s3://aarangog2proyectointegrador/zona trusted/data_imputed'

data_imputed.write.format('parquet').save(write_uri)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Generación de data_numeric

In [76]:
data_numeric = data_imputed.drop("country", "country_code", "region")
data_numeric.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['year', 'une_pop', 'une_gni', 'gghe-d', 'che_gdp', 'une_edu_spend', 'basic_water', 'doctors', 'bmi', 'age5-19thinness', 'age5-19obesity', 'alcohol', 'hepatitis', 'measles', 'polio', 'diphtheria', 'une_hiv', 'life_expect']

In [77]:
# Guardar en S3
write_uri='s3://aarangog2proyectointegrador/zona trusted/data_numeric'

data_numeric.write.format('parquet').save(write_uri)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Generación de data_standard

In [78]:
# Vectorizar las columnas numéricas
data_numeric = data_numeric.drop("life_expect")
input_cols = data_numeric.columns
assembler = VectorAssembler(inputCols=input_cols, outputCol="features_vec")
data_vec = assembler.transform(data_numeric)

# Aplicar escalado
scaler = StandardScaler(inputCol="features_vec", outputCol="scaled_features", withMean=True, withStd=True)
scaler_model = scaler.fit(data_vec)
data_scaled = scaler_model.transform(data_vec)

# Convertir el vector escalado en columnas individuales
data_array = data_scaled.withColumn("scaled_array", vector_to_array("scaled_features"))

# Extraer cada valor del array y renombrar como columna original
for i, c in enumerate(input_cols):
    data_array = data_array.withColumn(c, expr(f"scaled_array[{i}]"))

# Seleccionar solo las columnas originales, ahora estandarizadas
data_standard = data_array.select(input_cols)

data_standard.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+
|               year|             une_pop|            une_gni|             gghe-d|           che_gdp|      une_edu_spend|        basic_water|            doctors|                bmi|   age5-19thinness|     age5-19obesity|            alcohol|          hepatitis|            measles|              polio|         diphtheria|             une_hiv|
+-------------------+--------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+----------------

In [79]:
# Reagregar columnas categóricas
data_categoricas = data_imputed.select("region", "country", "country_code", "life_expect")
data_standard = data_standard.join(data_categoricas)

data_standard.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-----------------+------------------+--------------------+------+-------+------------+-----------+
|              year|             une_pop|            une_gni|            gghe-d|           che_gdp|      une_edu_spend|       basic_water|            doctors|                bmi|   age5-19thinness|     age5-19obesity|            alcohol|          hepatitis|            measles|            polio|        diphtheria|             une_hiv|region|country|country_code|life_expect|
+------------------+--------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+-----------

In [80]:
scaler.write().overwrite().save("s3://aarangog2proyectointegrador/zona refined/scaler")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
# Guardar en S3
write_uri='s3://aarangog2proyectointegrador/zona trusted/data_standard'

data_standard.write.format('parquet').save(write_uri)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Generación de data_prepared

In [82]:
# Indexar 'region' y obtener etiquetas
indexer = StringIndexer(inputCol="region", outputCol="region_index")
indexer_model = indexer.fit(data_standard)
region_labels = indexer_model.labels  # Lista de nombres reales de las regiones
data_indexed = indexer_model.transform(data_standard)

# One-hot encoding con dropLast=False
encoder = OneHotEncoder(inputCols=["region_index"], outputCols=["region_ohe"], dropLast=False)
encoder_model = encoder.fit(data_indexed)
data_encoded = encoder_model.transform(data_indexed)

# Convertir vector a array
data_with_array = data_encoded.withColumn("region_ohe_array", vector_to_array("region_ohe"))

# Crear columnas individuales con nombres reales de las regiones
for i, label in enumerate(region_labels):
    data_with_array = data_with_array.withColumn(f"{label}", col("region_ohe_array")[i])

# Eliminar columnas no necesarias
data_prepared = data_with_array.drop("region", "country", "country_code", "region_index", "region_ohe", "region_ohe_array")

# Mostrar resultados
data_prepared.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-----------------+------------------+--------------------+-----------+------+------+--------+---------------+---------------------+---------------+
|              year|             une_pop|            une_gni|            gghe-d|           che_gdp|      une_edu_spend|       basic_water|            doctors|                bmi|   age5-19thinness|     age5-19obesity|            alcohol|          hepatitis|            measles|            polio|        diphtheria|             une_hiv|life_expect|Europe|Africa|Americas|Western Pacific|Eastern Mediterranean|South-East Asia|
+------------------+--------------------+-------------------+------------------+------------------+-------------------+------------------+------------

In [83]:
# Guardar en S3
write_uri='s3://aarangog2proyectointegrador/zona trusted/data_prepared'

data_prepared.write.format('parquet').save(write_uri)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Generación de data_prepared_selected

In [84]:
columns_to_drop = ["Eastern Mediterranean", "South-East Asia", "Americas", "Europe", "Western Pacific"]
data_prepared_selected = data_prepared.drop(*columns_to_drop)

data_prepared_selected.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-----------------+------------------+--------------------+-----------+------+
|              year|             une_pop|            une_gni|            gghe-d|           che_gdp|      une_edu_spend|       basic_water|            doctors|                bmi|   age5-19thinness|     age5-19obesity|            alcohol|          hepatitis|            measles|            polio|        diphtheria|             une_hiv|life_expect|Africa|
+------------------+--------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------

In [85]:
# Guardar en S3
write_uri='s3://aarangog2proyectointegrador/zona trusted/data_prepared_selected'

data_prepared_selected.write.format('parquet').save(write_uri)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…